In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from  more_itertools import unique_everseen
import itertools
from datetime import datetime as dt
from urllib.request import *
import re

In [2]:
pd.options.mode.chained_assignment = None

In [55]:
headlinelinks=[]
headlinetitles=[]
timestamps=[]

url = ["https://www.reuters.com/news/archive/mergersNews?view=page&page=1&pageSize=10","https://www.reuters.com/news/archive/EUmergersNews?view=page&page=1&pageSize=10"]

for i in range(20):
    new_page = "page="+str(i)
    for j in url:
        new_url = j
        new_url = new_url.replace("page=1",new_page)
        html = urlopen(new_url)
        soup = bs(html,'html.parser')
        headlines = soup.find_all('div',class_='story-content')
        headlinelinks.extend( [x.find('a')['href'] for x in headlines] )
        headlinetitles.extend( [x.find('h3').text for x in headlines] )
        datetm = [x.find('span',class_='timestamp').text for x in headlines]
        datetm = [re.sub(' EDT','',x) for x in datetm]
        timestamps.extend(datetm)

In [56]:
removelist=['$','million','mln','']
headlinetitles = [x.replace("\t","") for x in headlinetitles]
headlinetitles = [x.replace("\n","") for x in headlinetitles]
#headlinetitles = list(unique_everseen(headlinetitles))
cleanedheadlinetitles = [re.sub("'s","",x) for x in headlinetitles]
cleanedheadlinetitles = [re.sub('[^a-zA-Z|\s]',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(million|mln|billion|bln)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s(a|to|be)\s',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('\s+',' ',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('BRIEF','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [re.sub('UPDATE','',x) for x in cleanedheadlinetitles]
cleanedheadlinetitles = [x.strip() for x in cleanedheadlinetitles]

In [58]:
newtimestamps = []
for item in timestamps:
    try:
        newtimestamps.append(dt.strptime(item,'%b %d %Y'))
    except:
        item = re.sub(r'(am|pm)',r' \1',item)
        currentime = dt.strptime(item,'%I:%M %p')
        currentdate = dt.today()
        currentdate = currentdate.replace(hour = currentime.hour, minute = currentime.minute)
        newtimestamps.append(currentdate)

In [59]:
headlinesframe=[]
headlinesframe = pd.DataFrame(cleanedheadlinetitles,columns=['titles'])
headlinesframe['link'] = pd.DataFrame(headlinelinks)
headlinesframe['datetime'] = pd.DataFrame(newtimestamps)
headlinesframe.sort_values('datetime',ascending=False,inplace=True)
headlinesframededup = headlinesframe.drop_duplicates(subset=['titles'], keep='first')

In [168]:
namelist = pd.read_excel('NameList.xlsx','ModNamesList')
namelistmod = namelist.iloc[:,1:]
searcharray = namelistmod.as_matrix()
newsearcharray=[]
for row in searcharray:
    newrow = [x.strip() for x in row if not pd.isnull(x)]
    newsearcharray.append(newrow)

In [172]:
def checknames(headlines):
    rowname = []
    names = []

    for new in headlines:
        words = new.split()
        s = " "
        newwords=[]    
        for i in range(int(len(words))):
            for j in range(1,5):
                newwords.append(s.join(words[i:i+j])) 
        newwords = list(set(newwords))
        newwords.sort(key = len,reverse=True)
    
        cos = []
        foundwords=[]
    
        for word in newwords:
        
            founds = [ix for ix, row in enumerate(newsearcharray) for iy, el in enumerate(row) if el.strip().lower() == word.lower()]          
            foundd = [rowd for ix,rowd in enumerate(foundwords) if rowd.strip().lower() == word.lower()]
        
            if len(foundd)!=0:
                continue
        
            if len(founds)!=0:
                cos.extend(founds)
                neww=[]
                w = word.split()
                for i in range(int(len(w))):
                    for j in range(1,5):
                        neww.append(s.join(w[i:i+j])) 
                neww = list(set(neww))
                foundwords.extend(neww)
                continue
        
        cos = list(set(cos))
        names.append([newsearcharray[i][0] for i in cos])
    return(names)

In [176]:
def check_mna(linknames):
    relevant = []
    wordlist = ["buy","acquire","acquisition","merge","merger","spinoff","buyout","lbo","stake","sale"]
    for link in linknames:
        founds = [ix for ix, row in enumerate(wordlist) if row in link.lower()]
        if len(founds)!=0:
            relevant.append(True)
        else:
            relevant.append(False)
    return(relevant)

In [174]:
companeslist = checknames(headlinesframededup['titles'])
relevantlist = check_mna(headlinesframededup['titles'])
headlinesframededup['companies']=companeslist  
headlinesframededup['relevant']=relevantlist  

In [175]:
headlinesframededup

,titles,link,datetime,companies,relevant
20,Brazil JBS resume operations at plants in Brazil,/article/jbs-operations/brazils-jbs-to-resume-...,2017-10-21 09:55:43.151972,[],False
30,Spain Rajoy says will sack Catalan government ...,/article/spain-politics-catalonia-rajoy/spains...,2017-10-21 07:58:43.152319,[],False
31,U S regulator clears Nutella maker acquisition...,/article/ferrara-candy-company-ma-ferrero-regu...,2017-10-21 05:36:43.152384,[],True
23,China says will guide private capital into hig...,/article/china-economy-investment/china-says-w...,2017-10-21 04:03:43.152164,[],False
48,Linde passes important percent threshold in Pr...,/article/linde-ma-praxair/linde-passes-importa...,2017-10-20 00:00:00.000000,[PRAXAIR INC],False
49,Germany will await prosecution ok before deliv...,/article/israel-corruption-germany/germany-wil...,2017-10-20 00:00:00.000000,[],False
60,Burger King operator BK Brasil files for IPO i...,/article/bk-brasil-ipo/update-1-burger-king-op...,2017-10-20 00:00:00.000000,[],False
52,Baker Hughes GE Co Q adj basic earnings per share,/article/brief-baker-hughes-a-ge-co-q3-adj-bas...,2017-10-20 00:00:00.000000,[BAKER HUGHES INC],False
54,Jamaica or bust Merkel launching crunch German...,/article/germany-politics/jamaica-or-bust-merk...,2017-10-20 00:00:00.000000,[],False
55,Russia Rosneft take control of Iraqi Kurdish p...,/article/mideast-crisis-iraq-kurds-rosneft/upd...,2017-10-20 00:00:00.000000,[],False


In [171]:
### for debug
news = ["PRESS DIGEST New York Times business news Oct","Google in talks to aquire Facebook labs"]
names=[]

for new in news:
    words = new.split()
    s = " "
    newwords=[]    
    for i in range(int(len(words))):
        for j in range(1,5):
            newwords.append(s.join(words[i:i+j])) 
    newwords = list(set(newwords))
    newwords.sort(key = len,reverse=True)
    
    cos = []
    foundwords=[]
    
    
    for word in newwords:
        
        founds = [ix for ix, row in enumerate(newsearcharray) for iy, el in enumerate(row) if el.strip().lower() == word.lower()]          
        foundd = [rowd for ix,rowd in enumerate(foundwords) if rowd.strip().lower() == word.lower()]
        
        if len(foundd)!=0:
            continue
        
        if len(founds)!=0:
            cos.extend(founds)
            neww=[]
            w = word.split()
            for i in range(int(len(w))):
                for j in range(1,5):
                    neww.append(s.join(w[i:i+j])) 
            neww = list(set(neww))
            foundwords.extend(neww)
            continue
        
    cos = list(set(cos))
    names.append([newsearcharray[i][0] for i in cos])
names


[['NEW YORK TIMES CO'], ['FACEBOOK INC', 'ALPHABET INC CL C']]

In [163]:
#save output
writer = pd.ExcelWriter('output3.xlsx')
headlinesframededup.to_excel(writer)